In [ ]:
import numpy as np
import os, cv2, pickle, random
import matplotlib.pyplot as plt

DATADIR = "D:\\Dataset\\chest_xray\\train"
CATEGORIES = ['PNEUMONIA', 'NORMAL']
IMG_SIZE = 200
training_data = []

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            img = os.path.join(path, img)
            img_array = cv2.imread(img)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training_data.append([new_array, class_num])
        except Exception as e:
            print(e)

random.shuffle(training_data)
print(len(training_data))

In [ ]:
X = []
y = []

for feature, label in training_data:
    X.append(feature)
    y.append(label)

X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

In [ ]:
pickle_out_X = open('X_orig.pickle', 'wb')
pickle.dump(X, pickle_out_X)
pickle_out_X.close()

pickle_out_y = open('y_orig.pickle', 'wb')
pickle.dump(y, pickle_out_y)
pickle_out_y.close()

In [3]:
# Creating Test Dataset

import numpy as np
import os, cv2, pickle, random
import matplotlib.pyplot as plt

DATADIR = "D:\\Dataset\\chest_xray\\test"
CATEGORIES = ['PNEUMONIA', 'NORMAL']
IMG_SIZE = 200
test_data = []

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            img = os.path.join(path, img)
            img_array = cv2.imread(img)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            test_data.append([new_array, class_num])
        except Exception as e:
            print(e)

random.shuffle(test_data)
print(len(test_data))

X_test = []
y_test = []

for feature, label in test_data:
    X_test.append(feature)
    y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_test.shape, y_test.shape)

624
(624, 200, 200, 3) (624,)


In [5]:
import pickle
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

pickle_in = open('X.pickle', 'rb')
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('y.pickle', 'rb')
y = pickle.load(pickle_in)
pickle_in.close()

X = X / 255.0

# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = define_model()
model.fit(X, y, batch_size = 32, epochs = 25, validation_data = (X_test[:522], y_test[:522]), shuffle = True)

Train on 8896 samples, validate on 522 samples
Epoch 1/25
8896/8896 [==============================] - 18s 2ms/step - loss: 0.3918 - acc: 0.8182 - val_loss: 3.0359 - val_acc: 0.8084
Epoch 2/25
8896/8896 [==============================] - 17s 2ms/step - loss: 0.1980 - acc: 0.9250 - val_loss: 3.6152 - val_acc: 0.7739
Epoch 3/25
8896/8896 [==============================] - 17s 2ms/step - loss: 0.1552 - acc: 0.9445 - val_loss: 3.5566 - val_acc: 0.7778
Epoch 4/25
8896/8896 [==============================] - 17s 2ms/step - loss: 0.1391 - acc: 0.9490 - val_loss: 2.8702 - val_acc: 0.8218
Epoch 5/25
8896/8896 [==============================] - 17s 2ms/step - loss: 0.1131 - acc: 0.9578 - val_loss: 2.4010 - val_acc: 0.8506
Epoch 6/25
8896/8896 [==============================] - 17s 2ms/step - loss: 0.1017 - acc: 0.9621 - val_loss: 3.4702 - val_acc: 0.7835
Epoch 7/25
8896/8896 [==============================] - 17s 2ms/step - loss: 0.0916 - acc: 0.9647 - val_loss: 2.9320 - val_acc: 0.8180
Epoch 8/

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test)  # evaluate the out of sample data with model
print(val_loss)  # model's loss (error)
print(val_acc)  # model's accuracy

In [ ]:
import pickle
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

pickle_in = open('X_orig.pickle', 'rb')
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('y_orig.pickle', 'rb')
y = pickle.load(pickle_in)
pickle_in.close()

X = X / 255.0

# define cnn model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(200, 200, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = define_model()
model.fit(X, y, batch_size = 32, epochs = 5, validation_split = 0.1, shuffle = True)

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold

pickle_in = open('X_orig.pickle', 'rb')
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('y_orig.pickle', 'rb')
y = pickle.load(pickle_in)
pickle_in.close()

X = X / 255.0

seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)


# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

scores = []

for train, test in kfold.split(X, y):
    model = define_model()
    model.fit(X[train], y[train], batch_size = 32, epochs = 3, shuffle = True)
    _, score = model.evaluate(X[test], y[test], verbose = 0)
    print('Accuracy:', score)
    scores.append(score)

print('Avg. Accuracy:', sum(scores)/len(scores))

D:\Anaconda Navigator\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda Navigator\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda Navigator\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda Navigator\envs\tensorflow\lib\site-packages\tensorflow\

Epoch 1/3
4694/4694 [==============================] - 12s 3ms/step - loss: 0.3015 - acc: 0.8775
Epoch 2/3
4694/4694 [==============================] - 9s 2ms/step - loss: 0.1382 - acc: 0.9478
Epoch 3/3
4694/4694 [==============================] - 9s 2ms/step - loss: 0.1138 - acc: 0.9582
Accuracy: 0.9559386968612671
Epoch 1/3
4694/4694 [==============================] - 9s 2ms/step - loss: 0.3200 - acc: 0.8671
Epoch 2/3
4694/4694 [==============================] - 9s 2ms/step - loss: 0.1469 - acc: 0.9431
Epoch 3/3
4694/4694 [==============================] - 9s 2ms/step - loss: 0.1111 - acc: 0.9555
Accuracy: 0.9080459765547537
Epoch 1/3
4694/4694 [==============================] - 9s 2ms/step - loss: 0.3513 - acc: 0.8409
Epoch 2/3
4694/4694 [==============================] - 8s 2ms/step - loss: 0.1594 - acc: 0.9350
Epoch 3/3
4694/4694 [==============================] - 9s 2ms/step - loss: 0.1036 - acc: 0.9574
Accuracy: 0.9712643680444623
Epoch 1/3
4694/4694 [===========================